We ignored all the sequence in the previous step, included a naive summary statistic as a feature. Trained linear regression, obtained half of the state of the art on this dataset. Not a bad start, eh?

Let's do something better, we want to capture the sequential statistics.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
# import seaborn as sns
plt.rcParams['figure.figsize'] = 15, 12

In [3]:
d = pd.read_csv('data/DREAM5_PBM_Data_TrainingSet.txt', sep='\t')

In [4]:
d.head()

,TF_Id,ArrayType,Sequence,Signal_Mean,Background_Mean,Signal_Median,Background_Median,Signal_Std,Background_Std,Flag
0,Cebpb,HK,AAAAAACAACAGGAGGGCATCATGGAGCTGTCCAGCCTGTGTGAAA...,1508.7972,264.8014,1557.5,183.0,393.2445,249.7754,0
1,Cebpb,HK,AAAAAACAGCCGGATCACAATTTTGCCGAGAGCGACCTGTGTGAAA...,13630.7324,371.0260,14858.0,245.0,6451.2436,445.4670,0
2,Cebpb,HK,AAAAAACGTCCGGTACACCCCGTTCGGCGGCCCAGCCTGTGTGAAA...,1523.0000,265.7928,1571.5,194.5,439.6682,211.7982,0
3,Cebpb,HK,AAAAAACTCTAGACCTTTAGCCCATCGTTGGCCAACCTGTGTGAAA...,4992.8461,430.4496,5150.0,233.0,1629.2899,507.5433,0
4,Cebpb,HK,AAAAAAGAACAACCGGATAACACCCTTACAGCACACCTGTGTGAAA...,3539.8117,449.4453,3823.5,226.5,1063.9028,551.7675,0


In [8]:
import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable

In [ ]:
# no gpu@ system change!